# Assignment 7 - Age of Abalone

## Nourhan El Zarif

## Import Data

In [1]:
import pandas as pd

file = 'https://library.startlearninglabs.uw.edu/DATASCI420/2019/Datasets/Abalone.csv'

In [2]:
data = pd.read_csv(file)
#data.head()

In [3]:
data['Sex'] = data['Sex'].astype('category').cat.codes
data.head()

,Sex,Length,Diameter,Height,Whole Weight,Shucked Weight,Viscera Weight,Shell Weight,Rings
0,2,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,2,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,0,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,2,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,1,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


In [4]:
data.shape

(4177, 9)

In [5]:
x = data.iloc[:, :8]
y = data.iloc[:, 8]

In [6]:
y.head(15)

0     15
1      7
2      9
3     10
4      7
5      8
6     20
7     16
8      9
9     19
10    14
11    10
12    11
13    10
14    10
Name: Rings, dtype: int64

### Convert target to Binary

In [7]:
y = y >10
y = y.astype(int)
y.head(15)

0     1
1     0
2     0
3     0
4     0
5     0
6     1
7     1
8     0
9     1
10    1
11    0
12    1
13    0
14    0
Name: Rings, dtype: int32

### Train Test Split

In [8]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, random_state = 1)

# Create SVM

In [9]:
from sklearn import svm, metrics
from sklearn.metrics import classification_report

cost = 0.9
gamma = 5

clf1 = svm.LinearSVC(C=cost).fit(x_train, y_train)
clf1.predict(x_test)
print("LinearSVC")
print(classification_report(clf1.predict(x_test), y_test))

for k in ('linear', 'rbf', 'poly'):
    clf = svm.SVC(gamma=gamma, kernel=k, C=cost).fit(x_train, y_train)
    clf.predict(x_test)
    print(k)
    print(classification_report(clf.predict(x_test), y_test))

LinearSVC
              precision    recall  f1-score   support

           0       0.89      0.76      0.82       315
           1       0.49      0.70      0.58       103

    accuracy                           0.75       418
   macro avg       0.69      0.73      0.70       418
weighted avg       0.79      0.75      0.76       418

linear
              precision    recall  f1-score   support

           0       0.92      0.77      0.84       325
           1       0.48      0.76      0.59        93

    accuracy                           0.77       418
   macro avg       0.70      0.76      0.71       418
weighted avg       0.82      0.77      0.78       418

rbf
              precision    recall  f1-score   support

           0       0.90      0.79      0.84       312
           1       0.54      0.75      0.63       106

    accuracy                           0.78       418
   macro avg       0.72      0.77      0.74       418
weighted avg       0.81      0.78      0.79       418

The polynomial kernel had the best f1 score, recall, and precision. Additionally, the accuracy the highest.

# Choosing Best Kernal / Hyperparameter

In [10]:
from sklearn.model_selection import GridSearchCV

parameters = {'kernel': ('linear', 'rbf', 'poly'), 'C' : [0.5, 1], 'gamma': [1, 5]}

svc = svm.SVC()
clf = GridSearchCV(svc, parameters).fit(x_train, y_train)

clf.best_params_

{'C': 1, 'gamma': 5, 'kernel': 'poly'}

## Metrics for Best Model

In [11]:
print(classification_report(clf.predict(x_test), y_test))

              precision    recall  f1-score   support

           0       0.90      0.79      0.84       310
           1       0.56      0.76      0.64       108

    accuracy                           0.78       418
   macro avg       0.73      0.77      0.74       418
weighted avg       0.81      0.78      0.79       418



The search grid model has the best metrics so far.

# SVR Model with Orginal Continuous Target

In [12]:
features = data.iloc[:, :8]
target = data.iloc[:, 8]

feature_train, feature_test, target_train, target_test = train_test_split(features, target, test_size = 0.1, random_state = 1)

In [13]:
kern = 'poly'
cost = 1
gam = 5


clf = svm.SVR(gamma = gam, kernel = kern, C = cost).fit(feature_train, target_train)
target_pred = clf.predict(feature_test)

In [14]:
from sklearn.metrics import mean_squared_error

mean_squared_error(target_test, target_pred)

4.771828266383733

In [15]:
from statistics import variance
variance(target_pred)

4.780449699414018

The SVR model was trained with the origional continuous target. The hyperparameers used were the same as the best classification model. The root mean squared error is 4.77, and the variance is 4.78